In [1]:
import pandas as pd
from datetime import datetime
from shapely.geometry import Point
import seaborn as sns
import matplotlib.pyplot as plt
import folium

In [2]:
# bring in Service fault codes
codes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')
codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7124 entries, 0 to 7123
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Published in CES 14602  7124 non-null   object
 1   Cummins Fault Code      7124 non-null   int64 
 2   Revision                7124 non-null   int64 
 3   PID                     7124 non-null   object
 4   SID                     7124 non-null   object
 5   MID                     7124 non-null   object
 6   J1587 FMI               7124 non-null   int64 
 7   SPN                     7124 non-null   int64 
 8   J1939 FMI               7124 non-null   int64 
 9   J2012 Pcode             7124 non-null   object
 10  Lamp Color              7124 non-null   object
 11  Lamp Device             7124 non-null   object
 12  Cummins Description     7124 non-null   object
 13  Algorithm Description   2005 non-null   object
dtypes: int64(5), object(9)
memory usage: 779.3+ KB


/home/chunderdamus/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
# Bring in on-board diagnostic data (12,821,626 entries!)
onboard = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
onboard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12821626 entries, 0 to 12821625
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Id       int64 
 1   Name     object
 2   Value    object
 3   FaultId  int64 
dtypes: int64(2), object(2)
memory usage: 391.3+ MB


In [4]:
# bring in fault code data (1,187,335 entries)
faults = pd.read_csv('../data/J1939Faults.csv', low_memory=False)
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   RecordID               1187335 non-null  int64  
 1   ESS_Id                 1187335 non-null  int64  
 2   EventTimeStamp         1187335 non-null  object 
 3   eventDescription       1126490 non-null  object 
 4   actionDescription      0 non-null        float64
 5   ecuSoftwareVersion     891285 non-null   object 
 6   ecuSerialNumber        844318 non-null   object 
 7   ecuModel               1122577 non-null  object 
 8   ecuMake                1122577 non-null  object 
 9   ecuSource              1187335 non-null  int64  
 10  spn                    1187335 non-null  int64  
 11  fmi                    1187335 non-null  int64  
 12  active                 1187335 non-null  bool   
 13  activeTransitionCount  1187335 non-null  int64  
 14  faultValue        

In [5]:
# Merge faults and onboard datasets
on_faults = (pd.merge(
    faults, 
    onboard, 
    left_on = 'RecordID', 
    right_on = 'FaultId')
    .drop(
    columns = [
    'Id', 
    'FaultId']
))
on_faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Name,Value
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,IgnStatus,False
1,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,EngineOilPressure,0
2,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,EngineOilTemperature,96.74375
3,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,TurboBoostPressure,0
4,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,...,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000,EngineLoad,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12821621,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,...,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,EngineCoolantTemperature,181.4
12821622,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,...,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,ParkingBrake,False
12821623,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,...,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,SwitchedBatteryVoltage,14.1
12821624,1248457,123906113,2020-03-06 14:14:13.000,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,...,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000,DistanceLtd,28606.65625


Filter The Dataset Per Client Request

In [6]:
# Filter down to only include full derates
full_derates = on_faults.loc[on_faults['spn'] == 5246]

# Get rid of the rows where the EquipmentID is more than 5 characters
full_derates = full_derates.loc[full_derates['EquipmentID'].astype(str).str.len() <= 5]

# Filter out faults where the light is going off
full_derates = full_derates.loc[on_faults['active'] == True].reset_index(drop = True)

full_derates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11035 entries, 0 to 11034
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RecordID               11035 non-null  int64  
 1   ESS_Id                 11035 non-null  int64  
 2   EventTimeStamp         11035 non-null  object 
 3   eventDescription       0 non-null      object 
 4   actionDescription      0 non-null      float64
 5   ecuSoftwareVersion     9233 non-null   object 
 6   ecuSerialNumber        8942 non-null   object 
 7   ecuModel               9182 non-null   object 
 8   ecuMake                9182 non-null   object 
 9   ecuSource              11035 non-null  int64  
 10  spn                    11035 non-null  int64  
 11  fmi                    11035 non-null  int64  
 12  active                 11035 non-null  bool   
 13  activeTransitionCount  11035 non-null  int64  
 14  faultValue             0 non-null      float64
 15  Eq

In [7]:
def extract_datetime(date_string):
    return datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f')

# apply the function to the 'datetime' column using apply() and a lambda function
full_derates['EventTimeStamp'].apply(lambda x: extract_datetime(x))

0       2015-02-21 12:10:51
1       2015-02-21 12:10:51
2       2015-02-22 19:44:55
3       2015-02-22 19:44:55
4       2015-02-23 05:05:44
                ...        
11030   2020-02-24 15:27:26
11031   2020-02-24 15:27:26
11032   2020-02-24 15:27:26
11033   2020-02-24 15:27:26
11034   2020-02-24 15:27:26
Name: EventTimeStamp, Length: 11035, dtype: datetime64[ns]

Filter Data For Fault codes that occur .5 miles from service centers

In [9]:
latlongtest = full_derates
def filter_by_distance(latlongtest):
    locations = [(-86.4347222, 36.0666667, 'a'), (-86.4438888, 35.5883333, 'b'), (-83.174722, 36.1950, 'c')]
    for location in locations:
        point = Point(location[1], location[0])
        dist_col_name = f'dist_from_location_{location[2]}'
        latlongtest[dist_col_name] = latlongtest.apply(lambda row: point.distance(Point(row['Longitude'], row['Latitude'])), axis=1)
        latlongtest[f'within_0.5mi_{location[2]}'] = latlongtest[dist_col_name] < 0.01
    # Rename the distance columns to remove the lat/long coordinates and replace with a,b, or c
    latlongtest = latlongtest.rename(columns={
        f"dist_from_location_{location[2]}" if i < 3 else f"dist_from_{location[2]}": f"dist_from_{location[2]}"
        for i, location in enumerate(locations)
    })
    return latlongtest

# Apply function to latlong test
filter_by_distance(latlongtest)

#Filter codes that occur less than 0.5 miles from the three service centers (within 0.5 miles = False)
mask = (latlongtest[['within_0.5mi_a', 'within_0.5mi_b', 'within_0.5mi_c']] == False).all(axis=1)
latlongtest_filtered = latlongtest[mask]

In [10]:
latlongtest_filtered

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,...,Longitude,LocationTimeStamp,Name,Value,dist_from_location_a,within_0.5mi_a,dist_from_location_b,within_0.5mi_b,dist_from_location_c,within_0.5mi_c
0,46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,...,-86.433425,2015-02-21 12:11:27.000,IgnStatus,True,173.241717,False,172.910311,False,171.044070,False
1,46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,...,-86.433425,2015-02-21 12:11:27.000,LampStatus,22527,173.241717,False,172.910311,False,171.044070,False
2,1919,1007751,2015-02-22 19:44:55.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,...,-86.434814,2015-02-22 19:46:27.000,IgnStatus,True,173.242863,False,172.911454,False,171.045226,False
3,1919,1007751,2015-02-22 19:44:55.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,...,-86.434814,2015-02-22 19:46:27.000,LampStatus,22527,173.242863,False,172.911454,False,171.045226,False
4,2090,1011009,2015-02-23 05:05:44.000,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,...,-74.087777,2015-02-23 05:08:23.000,IgnStatus,False,168.242781,False,167.936930,False,165.877716,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11030,1245454,122305096,2020-02-24 15:27:26.000,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,...,-86.434722,2020-02-24 15:28:02.000,CruiseControlSetSpeed,0,173.243092,False,172.911685,False,171.045451,False
11031,1245454,122305096,2020-02-24 15:27:26.000,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,...,-86.434722,2020-02-24 15:28:02.000,CruiseControlActive,False,173.243092,False,172.911685,False,171.045451,False
11032,1245454,122305096,2020-02-24 15:27:26.000,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,...,-86.434722,2020-02-24 15:28:02.000,EngineCoolantTemperature,77,173.243092,False,172.911685,False,171.045451,False
11033,1245454,122305096,2020-02-24 15:27:26.000,NaN,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,...,-86.434722,2020-02-24 15:28:02.000,ParkingBrake,True,173.243092,False,172.911685,False,171.045451,False


In [25]:
import plotly.express as px

# Create a dataframe for plotly
df = latlongtest_filtered

# Create the scatter plot
fig = px.scatter_mapbox(df, 
                        lat="Latitude", 
                        lon="Longitude", 
                        color_discrete_sequence=["red"], 
                        zoom=3.2,
                        hover_name="EquipmentID", 
                        hover_data=["LocationTimeStamp", "Name"])

# Customize the map style to show terrain
fig.update_layout(mapbox_style="stamen-terrain")
fig.show()
